In [47]:
def naver_kin(search,start_date,end_date,page_num=1):
    import time
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    import pandas as pd

    options=Options()
    options.add_experimental_option('detach',True)
    options.add_argument('--start-maximized')
    service=Service(ChromeDriverManager().install())

    driver=webdriver.Chrome(service=service, options=options)
    
    # search_date_length = (dt.datetime.strptime(end_date,"%Y.%m.%d")-dt.datetime.strptime(start_date,"%Y.%m.%d")).days
    # search = '부동산'
    # start_date = '2024.02.01'
    # end_date = '2024.02.08'
    res = [{'질문':""}]
    for page in range(1, page_num+1):
        url = f'https://kin.naver.com/search/list.naver?query={search}&section=qna&period={start_date}.|{end_date}.&sort=none&page={page}'

        driver.get(url)
        time.sleep(1)
        
        for list_num in range(1,11):
            tmp={}
            driver.find_element(By.XPATH,f'//*[@id="s_content"]/div[3]/ul/li[{list_num}]/dl/dt/a').click()
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[-1])
            tmp['질문'] = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/div[1]').text.replace('질문\n','')
            tmp['상세질문'] = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/div[3]').text.replace('\n','')
            tmp['답변개수'] = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[7]/div[1]/div/div/div[1]/span').text
            tmp['답변'] = [i.text for i in driver.find_elements(By.CLASS_NAME,'se-component-content')]
            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            if tmp['질문'] == res[-1]['질문']:
                break
            res.append(tmp)
    driver.quit()
    res = res[1:]
    df = pd.DataFrame(res)
    return df


In [46]:
df = naver_kin('부동산','2024.02.01','2024.02.08',10)
df

,질문,상세질문,답변개수,답변
0,월세 재계약 부동산에서 집주인 참석 없이 할 수 있나요,안녕하세요 월세 자취방을 2021년에 계약해서 지금까지 재계약 없이 묵시적 갱신으로...,1,"[공인중개사는 일방대리가 가능합니다.\n아마도, 임대인께서 해당 부동산에 위임을 맡..."
1,부동산 재계약,안녕하세요 서울에서 자취하고 있는 대학생입니다다름이 아니라 자취방 재계약 관련해서 ...,2,"[1. 첫 계약을 할 당시, 부동산 중개인이 보여준 이 집이 마음에 들었는데 알고보..."
2,다른부동산 매물,이번에 전세구하러 부동산에갔는데 다른동?의 매물을 하나 보여주시고 설명도 다해주시고...,3,"[네, 전세 구하는 목적으로 다른 부동산을 방문하신 것 같은데요. 때때로 다른 동의..."
3,부동산 매물 여러개,여러부동산에서 같은 매물이 여러개 올라와있는건 빨리 매매되라고 여러 부동산에 올려놓...,4,[다른 부동산에서도 경쟁적으로 올립니다.\n서로 계약하기 위해서 그렇게 경쟁적으로 ...
4,상속부동산 단독명의 또는 공동명의 중 유리한점??,안녕하세요~10년전 아빠가 돌아가시고 현 주택명의를 아빠에서 엄마 단독명의로 명의변...,1,[NAVER 지식iN 전문 상담세무사인 김기성 세무사입니다.\n 단독상속의 경우 상...
...,...,...,...,...
92,보증금 1억월세50에 살고 있는데부동산경매대출나올까요?,현재 청년 전월세보증금 대출 9천만원해서보증금 1억에 월세50에 살고 있는데만약에 ...,1,[부동산 경매 구입 및 대출 관련 질문 답변\n1. 부동산 경매 대출 가능성:\n9...
93,부동산 전세 중개수수료 질문,전세를 살고 있습니다2년을 살고 1회 갱신권을 사용하여 4년차를 살고 있습니다1회갱...,2,[정상 서류가 모두 오고 가면 전액 달라고 하고요\n대필 대서는 보통 각각 5~10...
94,상속 후 부동산 구매와 증여세 질문드립니다,사망하신 아버지의 예적금 1억과 보험금 3천을 첫째인 제가 동생에게 위임장을 받아 ...,1,[1. 상속을 받은 자가 상속재산을 부동산 구매에 사용할 수 있는 것은 가능합니다....
95,[세무] 법인 명의 부동산을 대표자 개인 명의로..,1인 법인을 없애기 전에 부동산(아파트 1채)을 대표자 개인(글쓴이) 명의로 돌리려...,1,[NAVER 지식iN 전문 상담세무사인 김기성 세무사입니다.\n 해당 부동산을 시가...


In [49]:
import pandas as pd

df.to_csv('네이버지식인_부동산_202402012024020810.csv')